In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import tkinter as tk
from tkinter import messagebox

# Function to fetch stock price
def get_stock_price():
    ab = entry.get().upper()

    if ab == 'EXIT':
        root.destroy()
        return

    if ab not in ticker_map:
        messagebox.showerror("Error", "Invalid ticker. Please try again.")
        return

    ticker = ticker_map[ab]
    exchange = 'NASDAQ'
    url = f'https://www.google.com/finance/quote/{ticker}:{exchange}'

    try:
        prices = []
        for i in range(3):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            class1 = "YMlKec fxKbKc"
            price = float(soup.find(class_=class1).text.strip()[1:].replace(",", ""))
            prices.append(price)
            result_label.config(text=f"Price for {ab}: {price}")
            root.update()
            time.sleep(5)
        avg_price = sum(prices) / len(prices)
        if ab in ticker_prices:
            ticker_prices[ab]['prices'].append(avg_price)
            if len(ticker_prices[ab]['prices']) > 1:
                initial_price = ticker_prices[ab]['prices'][0]
                current_price = ticker_prices[ab]['prices'][-1]
                performance = ((current_price - initial_price) / initial_price) * 100
                ticker_prices[ab]['performance'] = performance
        else:
            ticker_prices[ab] = {'prices': [avg_price], 'performance': 0}
        update_listbox()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Function to remove selected ticker from listbox
def remove_ticker():
    selected_ticker = listbox.curselection()
    if selected_ticker:
        listbox.delete(selected_ticker)

# Function to update listbox with latest prices and performance
def update_listbox():
    listbox.delete(0, tk.END)
    for ticker, data in ticker_prices.items():
        latest_price = data['prices'][-1]
        performance = data['performance']
        listbox.insert(tk.END, f"{ticker}: {latest_price:.2f} ({performance:.2f}%)")

# Initial ticker map and prices dictionary
ticker_map = {
    "AAPL": "AAPL",
    "GOOGL": "GOOGL",
    "MSFT": "MSFT",
    "TSLA": "TSLA",
    "META": "META",
    "NVDA": "NVDA"
}
ticker_prices = {}

# Tkinter GUI setup
root = tk.Tk()
root.title("Stock Portfolio Tracker")
root.geometry("485x450")
root.config(bg="Black")
root.resizable(width=False, height=False)

# Labels, Entry, and Buttons
label = tk.Label(root, text="Enter the ticker (or type 'exit' to quit):")
label.place(x=50, y=50)

entry = tk.Entry(root)
entry.place(x=280, y=50)

get_price_button = tk.Button(root, text="Get Price", command=get_stock_price)
get_price_button.place(x=225, y=100)

result_label = tk.Label(root, text="")
result_label.place(x=200, y=150)

remove_button = tk.Button(root, text="Remove Selected Ticker", command=remove_ticker)
remove_button.place(x=195, y=380)

# Listbox to display tickers with latest prices and performance
listbox = tk.Listbox(root)
update_listbox()  # Initial update
listbox.place(x=200, y=200)

root.mainloop()
